In [2]:
from problem1 import *
from problem2 import *
from problem3 import *
import pandas as pd
import numpy as np

In [56]:
import pandas as pd

blob = pd.read_pickle(r'C:\MFE\MFE Sem 3\CSE 426\CSE_426\Project 2\data\blobs_data.pkl')
blob.keys()

dict_keys(['tr_X', 'tr_y', 'te_X', 'te_y'])

In [67]:
model = SVMModel(blob['tr_X'],blob['tr_y'], 1, linear_kernel, 0.1)


In [72]:
x = np.array([model.train_X[:,6], model.train_X[:,89]]).T

K = linear_kernel(x, x)

alpha_vec = np.array([[model.alpha[0][6], model.alpha[0][89]]])
y_vec = np.array([[model.train_y[0][6], model.train_y[0][89]]])

In [59]:
np.dot(np.multiply(alpha_vec,y_vec), K) + model.b

array([[-0.97592554,  1.0514043 ]])

In [ ]:
K = np.zeros((model.train_X.shape[0], model.train_X[0])):
for i in model.train_X[:, 0]:

In [84]:
model = SVMModel(blob['tr_X'],blob['tr_y'], 1, linear_kernel, 0.1)
max_iters = 10 
record_every = 1 
max_passes = 1 
tol=1e-6
# Creating the lists
iter_num = []
duals = []
primals = []
models = []

# set dual variables to zero
#alpha = np.zeros(model.train_y.shape)
#b = 0

# begin iteration
for t in range(max_iters):
    passes = 0

    while passes < max_passes:
        num_changes = 0
        for i in range(model.m):
            
            if model.kernel_func.__name__ == 'linear_kernel':

                kkt = model.train_y[0,i]*((np.dot(np.multiply(model.alpha, model.train_y), linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])
            #    kkt= model.train_y[0,i]*((np.sum(np.multiply(model.alpha, model.train_y) @ linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])
            #    kkt= model.train_y[0,i]*((np.sum((model.alpha* model.train_y) @ linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])
            #    kkt = model.train_y[0,i]*(f_x(model.train_X, model.train_y, model.alpha, model.b, model.train_X[:,i], 1) - model.train_y[0,i])
            #    kkt = model.train_y[:,i]*((np.sum(np.multiply(model.alpha, model.train_y) * linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[:,i])
            else:

                kkt = model.train_y[0,i]*((np.dot(np.multiply(model.alpha, model.train_y), Gaussian_kernel(np.array([model.train_X[:,i]]), model.train_X).T) + model.b) - model.train_y[0,i])

            if (kkt < -tol and model.alpha[0,i] < model.C) or (kkt > tol and model.alpha[0,i] > 0):
                # Random initialize where i != j               
                list_for_randoms = [z for z in range(model.m)]

                list_for_randoms.remove(i)
                j = np.random.choice(list_for_randoms)
                # Traing Example
                x = np.array([model.train_X[:,i], model.train_X[:,j]]).T
                alpha_vec = np.array([[model.alpha[0][i], model.alpha[0][j]]])
                y_vec = np.array([[model.train_y[0][i], model.train_y[0][j]]])

                # Upper and Lower Bounds
                if y_vec[0,0] != y_vec[0,1]: 
                    H = min(model.C, model.C - (alpha_vec[0,0] - alpha_vec[0,1]))
                    L = max(0, -(alpha_vec[0,0] - alpha_vec[0,1]))

                else:
                    H = min(model.C, alpha_vec[0,0] + alpha_vec[0,1])
                    L = max(0, alpha_vec[0,0] + alpha_vec[0,1] - model.C)
                #if L==H:
                #    continue
                # Kernel Used
                if model.kernel_func.__name__ == 'linear_kernel':
                    K = linear_kernel(x, x)
                elif model.kernel_func.__name__ == 'Gaussian_kernel':
                    K = Gaussian_kernel(x, x)

                #g vector with g_1 and g_2 inside for efficiency
                g_vec = np.dot(np.multiply(alpha_vec, y_vec), K) + model.b

                # alpha_2 value
                #if K[0][0] + K[1][1] - 2*K[0][1] == 0:
                #    continue
                
                alpha_new = alpha_vec[0, 1] + ((y_vec[0, 1]*(g_vec[0][0] - y_vec[0, 0] - g_vec[0][1] + y_vec[0, 1]))
                                        /(K[0][0] + K[1][1] - 2*K[0][1]))

                #if alpha_new < 0:
                #    alpha_new =0

                # alpha_2 classification
                if alpha_new > H:
                    alpha_new_clipped = H

                elif alpha_new < L:
                    alpha_new_clipped = L

                else: 
                    alpha_new_clipped = alpha_new


                # Stopping if not meaningful change
                if np.abs(alpha_new_clipped-alpha_vec[0, 1]) < tol:

                    continue

                # Getting alpha_1 if meaningful change
                alpha_1_new = alpha_vec[0, 0] + (y_vec[0, 0]*y_vec[0, 1]) * (alpha_vec[0, 1] - alpha_new_clipped)

                # b primal
                # E vector (E_1, E_2)
                E = g_vec - y_vec
                b1 = model.b - E[0][0] - y_vec[0, 0]*(alpha_1_new - alpha_vec[0,0])*K[0][0] - y_vec[0, 1]*(alpha_new_clipped - alpha_vec[0, 1])*K[0][1]
                b2 = model.b - E[0][1] - y_vec[0, 0]*(alpha_1_new - alpha_vec[0,0])*K[0][1] - y_vec[0, 1]*(alpha_new_clipped - alpha_vec[0, 1])*K[1][1]

                # update model values
                # alphas
                model.alpha[0][i] = alpha_1_new
                model.alpha[0][j] = alpha_new_clipped

                # b primal
                if 0 < model.alpha[0][i] < model.C:
                    b_new = b1
                
                elif 0 < model.alpha[0][j] < model.C:
                    b_new = b2

                else:
                    b_new = (b1 + b2)/2

                model.b = b_new
                num_changes += 1

        if num_changes == 0:
            passes +=1
        else:
            passes = 0

    iter_num.append(t)
    duals.append(dual_objective_function(model.alpha, model.train_y, model.train_X, model.kernel_func, model.sigma))
    primals.append(primal_objective_function(model.alpha, model.train_y, model.train_X, model.b, model.C, model.kernel_func, model.sigma))
    models.append(vars(model))



KeyboardInterrupt: 

In [85]:
model.b

0.007702945429424235

In [82]:
predict(model, model.train_X)

array([-1,  1, -1, -1,  1,  1,  1,  1, -1, -1,  1, -1,  1, -1,  1,  1,  1,
       -1, -1,  1,  1,  1,  1, -1, -1,  1,  1, -1, -1,  1, -1, -1,  1, -1,
        1,  1, -1,  1,  1,  1,  1,  1, -1,  1, -1,  1, -1, -1, -1, -1, -1,
        1,  1,  1, -1,  1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,
       -1, -1,  1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1,  1, -1,  1, -1,
       -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1,  1, -1,  1, -1])

In [ ]:
decision_function()

In [80]:
import math
math.pow(10, -3)

0.001

In [73]:
model.b

-0.13163820959410277

In [40]:
def f_x(X, y, a, b, x, degree):
    predicted_value = 0.0
    # using polynomial kernel
    for k in range(X.shape[0]):
        predicted_value += (a[0,k]*y[0,k]*((X[:, k].T@x + 1)**degree))
    return predicted_value + b

In [77]:
model.train_y[0,i]*(f_x(model.train_X, model.train_y, model.alpha, model.b, model.train_X[:,i], 1) - model.train_y[0,i])

-0.3660487074655496

In [78]:
(f_x(model.train_X, model.train_y, model.alpha, model.b, model.train_X[:,i], 1) - model.train_y[0,i])

0.3660487074655496

In [65]:
model.alpha

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [30]:
model.train_X.shape

(2, 100)

In [51]:
model.train_y[0,i]*(f_x(model.train_X, model.train_y, model.alpha, model.b, model.train_X[:,i], 1) - model.train_y[0,i])

-1.0748116878371348

In [44]:
model.train_y[:,i]

array([-1], dtype=int64)

In [48]:
model.train_y[:,i]*(np.sum(np.multiply(model.alpha, model.train_y) * linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T)-model.train_y[:,i])

array([-1.])

In [46]:
model.alpha[:,i]

array([0.])

In [31]:
model.train_y[0,i]*((np.sum(np.multiply(model.alpha, model.train_y) * linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b) - model.train_y[0,i])

-1.0042847941385618

In [12]:
np.dot(np.multiply(model.alpha, model.train_y), linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T)

array([[-36.11151184]])

In [17]:
def f_x(X, y, a, b, x, degree):
    predicted_value = 0.0
    # using polynomial kernel
    for k in range(X.shape[0]):
        predicted_value += (a[k]*y[k]*((X[k, :].T@x + 1)**degree))
    return predicted_value + b

In [16]:
np.multiply(model.alpha, model.train_y)

array([[-3.55127255e-01,  0.00000000e+00, -5.83138495e-01,
        -3.01613781e-01,  5.31781920e-01,  0.00000000e+00,
         2.19284860e-01,  1.89837052e-01, -1.69103788e-01,
        -1.17689883e-01, -1.38777878e-17, -0.00000000e+00,
         0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  8.12125534e-01, -6.75457151e-01,
        -3.55480554e-01,  0.00000000e+00,  5.55111512e-17,
         3.50309026e-01,  3.25820527e-01, -0.00000000e+00,
        -4.72920754e-01,  0.00000000e+00,  1.14555490e-01,
        -1.86539553e-01, -1.79810283e-01,  3.01357910e-02,
        -0.00000000e+00, -7.30479941e-01,  0.00000000e+00,
        -1.00000000e+00,  8.19569549e-01, -5.55111512e-17,
        -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  4.60841427e-01,  8.05527984e-02,
        -6.91477181e-01,  1.59446036e-01, -0.00000000e+00,
         0.00000000e+00, -4.16333634e-17, -0.00000000e+00,
        -5.70819754e-01, -4.99861508e-01, -2.69969472e-0

In [14]:
linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T

array([[ 1.39939345],
       [-1.67662227],
       [ 1.21024948],
       [ 1.80518992],
       [-1.19675285],
       [-1.52493057],
       [-1.61707856],
       [-1.64633767],
       [ 1.5798916 ],
       [ 1.71293114],
       [-2.006546  ],
       [ 1.84679248],
       [-1.84445649],
       [ 1.75545409],
       [-1.56483984],
       [-1.45449057],
       [-1.53717665],
       [ 1.31647822],
       [ 1.55058214],
       [-1.92138979],
       [-1.54306482],
       [-1.76298827],
       [-1.70807075],
       [ 1.64458094],
       [ 1.42206042],
       [-2.21236543],
       [-1.63058592],
       [ 1.6220864 ],
       [ 1.57401593],
       [-1.79918717],
       [ 1.65407624],
       [ 1.30441083],
       [-2.00180053],
       [ 1.41816805],
       [-1.29086288],
       [-1.76936403],
       [ 1.9160753 ],
       [-1.95677251],
       [-1.85760455],
       [-1.76139296],
       [-1.6612102 ],
       [-1.86661533],
       [ 1.47910348],
       [-1.88327604],
       [ 1.68359556],
       [-1

In [10]:
model.b

0.004284794138566417

In [8]:
model.b

0.925268330628601

In [42]:
np.dot(np.multiply(model.alpha, model.train_y), linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T) + model.b - model.train_y[0,i]

array([[-32.82311169]])

In [35]:
kkt

array([[32.19008172]])

In [24]:
np.dot(np.multiply(model.alpha, model.train_y), linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X).T)

array([[-28.39396288]])

In [32]:
model.train_y[0,1]*((np.dot(np.multiply(model.alpha, model.train_y), linear_kernel(np.array([model.train_X[:,1]]).T,model.train_X).T) + model.b) - model.train_y[0,1])

array([[-1.]])

In [25]:
np.dot(np.multiply(model.alpha, model.train_y), linear_kernel(model.train_X[:,i],model.train_X))

array([-28.39396288])

In [20]:
linear_kernel(np.array([model.train_X[:,i]]).T,model.train_X)

array([[ 1.54669146, -1.50237635,  1.07995649,  1.6786708 , -0.96786672,
        -1.42299706, -1.49434298, -1.49345055,  1.34359446,  1.66639122,
        -1.82761631,  1.68549236, -1.74659288,  1.49840732, -1.22970759,
        -1.3542485 , -1.51850697,  1.25808608,  1.44461482, -1.94344538,
        -1.47049399, -1.72448601, -1.61316052,  1.60116312,  1.36663518,
        -1.98687814, -1.5391472 ,  1.58543586,  1.73879728, -1.62271371,
         1.51528678,  1.18040985, -1.89931607,  1.39295082, -1.16145714,
        -1.71572533,  1.89239966, -2.00683321, -1.76503002, -1.70339688,
        -1.58155481, -1.68750317,  1.4072979 , -1.94278708,  1.60457425,
        -1.49029034,  1.84102612,  2.00930981,  1.24665178,  1.40744218,
         1.45399353, -1.63276834, -1.56846547, -1.68211646,  1.63310927,
        -1.53157393,  1.43335421, -1.09578637,  1.60772574,  1.65069868,
         1.63077156, -1.74338833, -1.38056947, -1.73847096, -2.04971763,
        -1.30804394,  1.45182435, -1.9787448 ,  1.3

In [43]:
model.alpha

array([[ 7.64010135e-01,  0.00000000e+00,  5.87958371e-01,
         0.00000000e+00,  7.79729455e-01,  5.16675751e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.32955373e-02,  0.00000000e+00,  2.22909040e-01,
         1.13669060e-01,  4.85722573e-17,  1.20462462e-01,
         7.01009308e-01,  6.57968120e-01,  1.00000000e+00,
         6.71081912e-01,  0.00000000e+00,  3.30673845e-01,
        -2.77555756e-17,  5.45981067e-01,  4.23732975e-02,
         2.43508863e-01,  0.00000000e+00,  0.00000000e+00,
         1.74626938e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.95519174e-02,  2.77555756e-17,  2.54924659e-01,
         0.00000000e+00,  0.00000000e+00,  2.77329358e-01,
         3.67669605e-01,  0.00000000e+00,  0.00000000e+00,
         2.09318396e-01,  1.36328673e-01,  0.00000000e+00,
         4.43379367e-01,  1.00000000e+00,  5.46466340e-0

In [17]:
model.train_y[0,5]*((np.dot(np.multiply(model.alpha, model.train_y), linear_kernel(model.train_X[:,5], model.train_X))+model.b) - model.train_y[:,5])

array([-1.])

In [ ]:
np.dot(np.multiply(alpha_vec,y_vec), K) + model.b

In [ ]:
alpha = np.array

In [22]:
x = np.array([model.train_X[:,2], model.train_X[:,55]]).T
x

array([[ 0.60754434, -0.93602521],
       [ 0.85371564, -1.01201771]])

In [80]:
linear_kernel(model.train_X[:,2], model.train_X)

array([ 0.3309753 ,  0.38183647,  4.09976594,  0.36346719,  0.05359556,
       -1.78144625,  3.76525455,  0.36645388, -0.11002703, -0.25776056,
        0.19797236,  2.98134575, -0.4353054 , -0.28926887, -0.22191685,
        3.96834895,  3.93489895,  0.78591284,  0.34539171, -3.94460408,
       -2.85554575,  0.14201936, -0.05059693, -0.31451422, -0.8085973 ,
        3.95687104,  0.12557301,  2.91256744, -0.09890696,  3.29545653,
       -0.25210229, -0.18606368, -0.25553085,  0.06396623,  0.27115442,
        2.64267475, -1.142959  , -0.34910926,  1.18218049,  3.99800362,
        0.35455128, -0.32343196,  0.24880788, -2.59623929,  0.42959706,
        2.20273272,  0.34075337,  0.24343527,  2.47532118, -0.12660701,
        1.25775085, -0.37982381,  1.59577122, -3.5545315 , -0.41766605,
       -1.66192951,  0.14671119,  0.38102679, -3.21395132,  0.17270493,
        3.73441283, -0.23509671, -0.01539285,  1.76871935, -0.65397568,
        0.28935923, -4.06163269, -0.35357458, -2.10314969,  0.30

In [81]:
linear_kernel(np.array([model.train_X[:,2]]), model.train_X)

ValueError: shapes (2,1) and (2,100) not aligned: 1 (dim 1) != 2 (dim 0)

In [23]:
model.alpha

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [12]:
max(model.alpha[0])

1.0

In [13]:
min(model.alpha[0])

-1.734723475976807e-18

In [135]:
iter_num

[]

In [1]:
from problem1 import *
from problem2 import *
from problem3 import *
import pandas as pd
import numpy as np

trained = pd.read_pickle(r'C:\MFE\MFE Sem 3\CSE 426\CSE_426\Project 2\data\trained_model_blobs_Gaussian_kernel.pkl')

In [2]:
xrange = np.linspace(trained.train_X[0, :].min(), trained.train_X[0, :].max(), 100)
yrange = np.linspace(trained.train_X[1, :].min(), trained.train_X[1, :].max(), 100)
grid = [[decision_function(trained.alpha, trained.train_y, trained.train_X, trained.b, trained.kernel_func, trained.sigma, np.array([xr, yr])[:, np.newaxis])
                            for xr in xrange] for yr in yrange]

In [14]:
grid[0][0].shape

(100, 100)

In [22]:
trained.alpha

array([[0.99999901, 1.        , 0.99999961, 0.99999936, 1.        ,
        0.99999951, 1.        , 1.        , 1.        , 0.99999919,
        1.        , 1.        , 0.99999861, 1.        , 1.        ,
        1.        , 1.        , 0.99999958, 0.99999985, 1.        ,
        0.99999984, 1.        , 0.99999974, 1.        , 1.        ,
        0.99999982, 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 0.99999913, 0.99999895,
        0.99999993, 1.        , 1.        , 0.99999943, 0.99999969,
        0.99999932, 0.99999948, 1.        , 1.        , 0.99999983,
        1.        , 1.        , 0.99999982, 1.        , 1.        ,
        0.99999959, 1.        , 1.        , 0.99999812, 1.        ,
        0.9999992 , 1.        , 0.99999893, 0.99999999, 1.        ,
        1.        , 0.99999952, 1.        , 1.        , 0.99999971,
        0.99999927, 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.  

In [70]:
margin = (model.alpha[0][0]*linear_kernel(model.train_X[:,0], model.train_X[:,0]) + model.b)

-0.0

In [87]:
model.alpha

array([[0.58895634, 0.19320143, 0.41853519, 0.24637686, 0.41441496,
        0.08125328, 0.3643159 , 0.19926878, 0.30362285, 0.25450739,
        0.16618472, 0.3591592 , 0.20432465, 0.06850671, 0.27660162,
        0.30148788, 0.117666  , 0.29963977, 0.4334626 , 0.21405454,
        0.24515952, 0.28415141, 0.28847491, 0.40671835, 0.45769339,
        0.02921965, 0.16921848, 0.39018434, 0.15794722, 0.23193672,
        0.19320143, 0.39665965, 0.33884146, 0.48259342, 0.38312227,
        0.09093963, 0.10980852, 0.135395  , 0.06687377, 0.37356797,
        0.21064561, 0.09307643, 0.18817416, 0.30087085, 0.12803263,
        0.24637686, 0.28160198, 0.20577312, 0.25006489, 0.39165758,
        0.23622674, 0.03671709, 0.19154311, 0.09927758, 0.37089844,
        0.20793442, 0.304392  , 0.39018434, 0.2330284 , 0.28123094,
        0.16662785, 0.24879968, 0.52469228, 0.266099  , 0.31405299,
        0.43886509, 0.09085246, 0.21453337, 0.16436567, 0.33172054,
        0.27625477, 0.25006489, 0.40663649, 0.07

In [88]:
model.b

-0.15980347611071477

In [77]:
checks = [
    model.alpha[0][0] < 0,
    model.alpha[0][0] > model.C,
    #np.dot(model.alpha, model.train_y.T) != 0,
    (model.train_y[0,0]*margin < 1 and model.alpha[0,0] == 0),
    #model.alpha[0,i]*(model.train_y[0,i]*margin - 1 + eps) == 0

]

In [78]:
checks

[False, False, True]

In [79]:
any(checks)

True

In [71]:
(model.train_y[0,0]*margin >= 1 and model.alpha[0,0] == 0)

array([False])

In [68]:
linear_kernel(model.train_X[:,0], model.train_X[:,0])

1.5466914615723717

In [34]:
model.alpha

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [35]:
iter_num

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [36]:
models

[{'train_X': array([[ 1.16466313, -0.85136678,  0.60754434,  1.01140854, -0.44558325,
          -0.86205304, -0.89143236, -0.86423715,  0.6904045 ,  1.07374723,
          -1.06481792,  0.98529007, -1.08212632,  0.77566879, -0.52702774,
          -0.8175529 , -0.99939957,  0.79013893,  0.87295672, -1.31960575,
          -0.91973039, -1.1197026 , -0.99546646,  1.03286432,  0.86537465,
          -1.13029204, -0.94901453,  1.02831266,  1.30860331, -0.92990649,
           0.89128167,  0.68029733, -1.18022063,  0.90962759, -0.66285032,
          -1.10048918,  1.24511478, -1.38671641, -1.09912637, -1.08839767,
          -0.98777954, -0.97092239,  0.87812778, -1.35220668,  1.00373873,
          -0.92776222,  1.17339146,  1.48312316,  0.68637814,  0.92625906,
           0.90227196, -0.89650296, -1.03671097, -1.15218614,  0.98488907,
          -0.93602521,  0.80634259, -0.74821928,  0.96608659,  1.07963598,
           1.00090713, -1.21153725, -0.93410179, -1.06142179, -1.5036788 ,
          -0.7

In [37]:
model.alpha

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [38]:
d = np.array([1,1])

In [39]:
len(d.shape)

1

In [40]:
margin = model.train_y[0,i]*(model.alpha[0][i]*Gaussian_kernel(np.array([model.train_X[:,i]]), np.array([model.train_X[:,i]]), model.sigma) + model.b)
eps = max(0, 1-margin)

In [41]:
Gaussian_kernel(np.array([model.train_X[:,1]]), np.array([model.train_X[:,1]]), model.sigma)

array([1.])

In [42]:
np.array([model.train_X[:,1]]).shape

(1, 2)

In [43]:
np.exp(-(euclidean_distances(np.array([model.train_X[:,1]]),np.array([model.train_X[:,1]]))**2)/(2*model.sigma**2))

array([[1.]])

In [44]:
margin

array([-0.])

In [45]:
model.b

0

In [46]:
model.alpha[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [47]:
y = model.train_y[0, 10]
z = y*(model.alpha[0][10]*linear_kernel(model.train_X[:,10], model.train_X[:,10]) + model.b)
#z_error = model.alpha[0][10]*linear_kernel(model.train_X[:,10], model.train_X[:,10])

eps = max(0, 1-y*z)

In [48]:
y*z

0.0

In [49]:
eps

1.0

In [50]:
eps

1.0

In [51]:
y*z - 1 + eps

0.0

In [52]:
model.alpha[0][1] = 1

In [53]:
model.alpha

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [54]:
models

[{'train_X': array([[ 1.16466313, -0.85136678,  0.60754434,  1.01140854, -0.44558325,
          -0.86205304, -0.89143236, -0.86423715,  0.6904045 ,  1.07374723,
          -1.06481792,  0.98529007, -1.08212632,  0.77566879, -0.52702774,
          -0.8175529 , -0.99939957,  0.79013893,  0.87295672, -1.31960575,
          -0.91973039, -1.1197026 , -0.99546646,  1.03286432,  0.86537465,
          -1.13029204, -0.94901453,  1.02831266,  1.30860331, -0.92990649,
           0.89128167,  0.68029733, -1.18022063,  0.90962759, -0.66285032,
          -1.10048918,  1.24511478, -1.38671641, -1.09912637, -1.08839767,
          -0.98777954, -0.97092239,  0.87812778, -1.35220668,  1.00373873,
          -0.92776222,  1.17339146,  1.48312316,  0.68637814,  0.92625906,
           0.90227196, -0.89650296, -1.03671097, -1.15218614,  0.98488907,
          -0.93602521,  0.80634259, -0.74821928,  0.96608659,  1.07963598,
           1.00090713, -1.21153725, -0.93410179, -1.06142179, -1.5036788 ,
          -0.7

In [55]:
iter_num

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [56]:
models

[{'train_X': array([[ 1.16466313, -0.85136678,  0.60754434,  1.01140854, -0.44558325,
          -0.86205304, -0.89143236, -0.86423715,  0.6904045 ,  1.07374723,
          -1.06481792,  0.98529007, -1.08212632,  0.77566879, -0.52702774,
          -0.8175529 , -0.99939957,  0.79013893,  0.87295672, -1.31960575,
          -0.91973039, -1.1197026 , -0.99546646,  1.03286432,  0.86537465,
          -1.13029204, -0.94901453,  1.02831266,  1.30860331, -0.92990649,
           0.89128167,  0.68029733, -1.18022063,  0.90962759, -0.66285032,
          -1.10048918,  1.24511478, -1.38671641, -1.09912637, -1.08839767,
          -0.98777954, -0.97092239,  0.87812778, -1.35220668,  1.00373873,
          -0.92776222,  1.17339146,  1.48312316,  0.68637814,  0.92625906,
           0.90227196, -0.89650296, -1.03671097, -1.15218614,  0.98488907,
          -0.93602521,  0.80634259, -0.74821928,  0.96608659,  1.07963598,
           1.00090713, -1.21153725, -0.93410179, -1.06142179, -1.5036788 ,
          -0.7

In [57]:
KKTs = [
    model.train_y[0][10]*(model.alpha[0][10]*linear_kernel(model.train_X[:,10], model.train_X[:,10]) + model.b) - 1 <= 0
]

In [58]:
model.alpha[0][10]*linear_kernel(model.train_X[:,10], model.train_X[:,10])

0.0

In [59]:
KKTs

[True]

In [60]:
K[1,1]



2.3956535421647374

In [61]:
K

array([[ 3.2632457 , -2.78099446],
       [-2.78099446,  2.39565354]])

In [62]:
model.train_y[0][10]*(model.alpha[0][10]*linear_kernel(model.train_X[:,10], model.train_X[:,10]) + model.b) - 1

-1.0

In [63]:
x = np.array([model.train_X[:,10], model.train_X[:,18]]).T
x

array([[-1.06481792,  0.87295672],
       [-1.34684031,  0.98105428]])

In [64]:
model.train_X[:,18]

array([0.87295672, 0.98105428])

In [65]:
model.train_X

array([[ 1.16466313, -0.85136678,  0.60754434,  1.01140854, -0.44558325,
        -0.86205304, -0.89143236, -0.86423715,  0.6904045 ,  1.07374723,
        -1.06481792,  0.98529007, -1.08212632,  0.77566879, -0.52702774,
        -0.8175529 , -0.99939957,  0.79013893,  0.87295672, -1.31960575,
        -0.91973039, -1.1197026 , -0.99546646,  1.03286432,  0.86537465,
        -1.13029204, -0.94901453,  1.02831266,  1.30860331, -0.92990649,
         0.89128167,  0.68029733, -1.18022063,  0.90962759, -0.66285032,
        -1.10048918,  1.24511478, -1.38671641, -1.09912637, -1.08839767,
        -0.98777954, -0.97092239,  0.87812778, -1.35220668,  1.00373873,
        -0.92776222,  1.17339146,  1.48312316,  0.68637814,  0.92625906,
         0.90227196, -0.89650296, -1.03671097, -1.15218614,  0.98488907,
        -0.93602521,  0.80634259, -0.74821928,  0.96608659,  1.07963598,
         1.00090713, -1.21153725, -0.93410179, -1.06142179, -1.5036788 ,
        -0.7343002 ,  0.77392239, -1.36245897,  0.9